In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
#from scipy.sparse import hstackb
import bs4

In [3]:

#Bringing the datasets
train=pd.read_csv('Devex_train.csv',encoding='latin-1')
test=pd.read_csv('Devex_test_questions.csv',encoding='latin-1')
sample_sub=pd.read_csv('Devex_submission_format.csv')    #,encoding='latin-1')
#train = pd.read_csv('../input/train.csv').fillna(' ')
#test = pd.read_csv('../input/test.csv').fillna(' ')

train_text = train['Text']
test_text = test['Text']
all_text = pd.concat([train_text, test_text])

In [ ]:
train['Text'] = train['Text'].apply(lambda x: bs4.BeautifulSoup(x, 'lxml').get_text())
#train.head()

In [8]:
class_names = target_variables=list(sample_sub.columns.values)[1:]
train = train.reindex(columns=train.columns.tolist() + target_variables)
#filling the NaN values
train=train.select_dtypes(include=['object','float']).fillna('None')

In [ ]:
from bs4 import BeautifulSoup
cleantext = BeautifulSoup(raw_html, "lxml").text


In [10]:
#Transforming the target variables into a useful format
train["targets"]=train["Label 1"].apply(lambda x: x[:5] )
targets=pd.get_dummies(train["targets"])
#targets.select_dtypes(include=['object']).fillna(0)
for column in train.columns[4:15]:
    train["targets"]=train[column].apply(lambda x: x[:5] if (type(x)==str) else "None")
    new_targets=pd.get_dummies(train["targets"])
    diff=list(set(list(targets.columns))-set(list(new_targets.columns)))
    new_targets[diff]=targets[diff]*0
    #print(len(new_targets.columns))
    targets=targets + new_targets
    #print(targets["3.b.2"][0])
    if column=="Label 12":
        break
#targets

In [11]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 27),
    max_features=50000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

KeyboardInterrupt: 

In [ ]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = target[class_name]
    classifier = LogisticRegression(C=0.1, solver='sag')

    cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

submission.to_csv('submission.csv', index=False)